## Main ideas

* Create zero-shot baseline
* Train XLM-R on one language and zero-shot to another
* Create learning curves to see how much labelled data we need to beat the baseline

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# For HF machines
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Setup

In [3]:
# from huggingface_hub import notebook_login

# notebook_login()

## Imports

In [39]:
from datasets import load_dataset, get_dataset_config_names, load_metric, ClassLabel, Features
from sklearn.metrics import mean_absolute_error
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline
import numpy as np
from IPython.display import display, HTML
import pandas as pd

## Load data

In [36]:
dataset_name = "amazon_reviews_multi"
langs = get_dataset_config_names(dataset_name)
langs

['all_languages', 'de', 'en', 'es', 'fr', 'ja', 'zh']

In [37]:
german_dataset = load_dataset(path=dataset_name, name="de")
german_dataset

Reusing dataset amazon_reviews_multi (/data/.cache/hf/datasets/amazon_reviews_multi/de/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [38]:
german_dataset["train"][0]

{'review_id': 'de_0203609',
 'product_id': 'product_de_0865382',
 'reviewer_id': 'reviewer_de_0267719',
 'stars': 1,
 'review_body': 'Armband ist leider nach 1 Jahr kaputt gegangen',
 'review_title': 'Leider nach 1 Jahr kaputt',
 'language': 'de',
 'product_category': 'sports'}

In [ ]:
import random
import pandas as pd
from datasets import ClassLabel
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    "Taken from https://github.com/huggingface/notebooks/blob/master/examples/text_classification.ipynb"
    
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(german_dataset["train"])

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,de_0428326,product_de_0240520,reviewer_de_0678246,4,"Tisch etwas wackelig ,aber sonst zufriedenstellend",Sitzgruppe,de,lawn_and_garden
1,de_0280733,product_de_0829653,reviewer_de_0589657,3,"Ich trage sie beim Fußball. Sie tragen sich sehr bequem , schützen tun sie auch , aber sie verrutschen leicht . Das war ich so von meinen vorigen nicht gewohnt",Bequem aber rutschen leicht,de,sports
2,de_0865702,product_de_0580556,reviewer_de_0290736,4,"Schnelle Lieferung und gutes Aussehen. Wasserstandsanzeige unzureichend, Stift bleibt haengen.",Die Bewässerung der Pflanzen in den Sommermonaten könnte noch nicht getestet werfen,de,lawn_and_garden
3,de_0324642,product_de_0658991,reviewer_de_0277583,4,Am 4 april 2018 gekauft 16 Oktober ist der radio kaputt Update Habe jetzt ein neues radio bekommen bin jetzt zufrieden hoffe das es jetzt besser geht Kabel haben eine neuere Version und das radio auch,Super,de,wireless
4,de_0587772,product_de_0406394,reviewer_de_0178939,1,Zu teuer zu schlecht verarbeitet zweimal benutzt Schrott,Schrott,de,wireless
5,de_0165078,product_de_0503857,reviewer_de_0795147,1,Der Wein war geöffnet und gekippt,Schlecht,de,other
6,de_0299847,product_de_0455492,reviewer_de_0182181,1,Leider sticht der Punkt bei dem der Fingerabdruck in etwa ist sehr raus.,S10 nicht zu empfehlen,de,wireless
7,de_0783252,product_de_0681376,reviewer_de_0106943,5,Diese Hülle macht einen noch hochwertigeren Eindruck. Ist allerdings schon ein robust gebautes Teil. Sieht halt auch schon etwas martialischer aus. Das iPhone X schaut halt dann nicht mehr so filigran aus. Aber für mich genau richtig weil ich das Telefon schon sehr beanspruche und ich nicht ständig darauf achten will.,Sehr Stabil,de,wireless
8,de_0910019,product_de_0347563,reviewer_de_0553149,2,nach zu später lieferung und einigen schwierigkeiten bei der rücksendung ging am ende doch alles glatt. Alles sehr kompliziert,Verzögerte Rücksendung,de,home_improvement
9,de_0783482,product_de_0089593,reviewer_de_0177254,2,"Leider war ich von der Rosa/Marmor Optik schon etwas enttäuscht, die schon seeeehr gedruckt aussieht (wenn man versteht was ich damit meine) und zum anderen habe ich es auf eine Handyhülle geklebt wo es sehr gut hielt, als ich aber die Hülle wechselte, fiel sie sofort von der neuen ab und hielt auch sonst nirgendwo mehr dran. Also leider ein Flop Produkt",Hält leider nur kurz und die Optik sieht sehr gedruckt aus..,de,wireless


In [40]:
german_dataset.set_format("pandas")
german_df = german_dataset["train"][:]
german_df.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,de_0203609,product_de_0865382,reviewer_de_0267719,1,Armband ist leider nach 1 Jahr kaputt gegangen,Leider nach 1 Jahr kaputt,de,sports
1,de_0559494,product_de_0678997,reviewer_de_0783625,1,In der Lieferung war nur Ein Akku!,EINS statt ZWEI Akkus!!!,de,home_improvement
2,de_0238777,product_de_0372235,reviewer_de_0911426,1,"Ein Stern, weil gar keine geht nicht. Es hande...",Achtung Abzocke,de,drugstore
3,de_0477884,product_de_0719501,reviewer_de_0836478,1,"Dachte, das wären einfach etwas festere Binden...",Zu viel des Guten,de,drugstore
4,de_0270868,product_de_0022613,reviewer_de_0736276,1,Meine Kinder haben kaum damit gespielt und nac...,Qualität sehr schlecht,de,toy


In [41]:
sample = german_df.sample(n=10, random_state=42)
display(HTML(sample.to_html()))

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
119737,de_0970901,product_de_0712478,reviewer_de_0308094,3,Ist ok ...blondierung quillt schnell auf,Ok,de,beauty
72272,de_0042217,product_de_0734686,reviewer_de_0904358,2,Kein typischer Geruch oder Geschmack von einem Ghee! Ich würde es nicht wieder kaufen oder weiter empfehlen. Konkurrenz Produkt fand ich besser.,Kein typischer Geruch oder Geschmack von einem Ghee !,de,grocery
158154,de_0278932,product_de_0388890,reviewer_de_0940030,4,Dieses Buch hat mir sehr geholfen mit dem ersten Schlupf und der weiteren Aufzucht. Kann ich nur weiter empfehlen.,Sehr hilfreich,de,book
65426,de_0737352,product_de_0560586,reviewer_de_0632435,2,"super Schale, wunderschön, gutes Produkt ABER Der Saugnapf geht von der Schale runter, da die Maße des Saugnapf Ringes nicht passen. Man muss aufpassen dass man den nicht dauernd neu aufsetzen muss.",der Saugnapf hält nicht,de,baby_product
30074,de_0455430,product_de_0375951,reviewer_de_0482228,1,"Artikel ist niemals angekommen, habe ihn aber bezahlt! Und dann steht noch dort ich hätte unterschrieben, als er angeblich angekommen sei! null Sterne! Unglaublich 😒",Artikel ist niemals angekommen!!,de,book
23677,de_0007108,product_de_0230566,reviewer_de_0672101,1,habe das headset seit 2 jahren es ist einfach ein rauschen gekommen wenn ich rede einfach so man kann mich garnicht mehr verstehen,Mikrofon einfach Kaputt gegen ein rausch wenn ich rede,de,video_games
134858,de_0990930,product_de_0723421,reviewer_de_0677949,4,"War mein erster Versuch mit einem Seifenstück als Shampoo und ist etwas gewöhnungsbefürftig. Hatte am Anfang das Gefühl, dass die Haare nicht richtig sauber werden aber eine Freundin hat mir dann eine andere Methode zum Auftragen gezeigt mit der es ganz gut klappt. Besser für die Umwelt ist es aber allemals!",Ein Versuch wert,de,beauty
176418,de_0902392,product_de_0985447,reviewer_de_0695647,5,"Immer wieder gern zu spielen, viel Spaß und auch sehr spannend. Vorsicht: macht süchtig",Spiel,de,other
132467,de_0537198,product_de_0386242,reviewer_de_0590416,4,"Die Werkzeuge sind von guter Qualität, sie machen absolut das, was sie sollen. Allerdings war ein Schraubendreher durch den Versand leicht verbogen, daher nur 4 von 5 Sternen.","Tolles Set, Werkzeuge guter Qualität",de,electronics
4082,de_0626877,product_de_0410155,reviewer_de_0752644,1,"Sehr enttäuschend! Neu? Verdreckt, kaputt und stinkend kam die Kühlbox! Mehr als schade! Geht sofort zurück, dafür müsste ich es noch nicht mal aus dem Karton nehmen! Der Stern würde nur gegeben, weil diese Bewertung ohne nicht geht!",Sauer!!,de,automotive


In [42]:
german_df["product_category"].value_counts()

home                        26063
wireless                    19964
sports                      13748
home_improvement            12408
apparel                     10178
toy                          9781
pc                           8577
drugstore                    8075
lawn_and_garden              7426
beauty                       7162
electronics                  7114
other                        6460
furniture                    6334
kitchen                      5787
automotive                   5321
pet_products                 5028
book                         4927
office_product               4343
baby_product                 4070
shoes                        3568
luggage                      3256
digital_video_download       2970
personal_care_appliances     2836
grocery                      2737
digital_ebook_purchase       2720
jewelry                      2380
camera                       1906
watch                        1706
video_games                  1219
industrial_sup

In [43]:
german_df["stars"].value_counts()

1    40000
2    40000
3    40000
4    40000
5    40000
Name: stars, dtype: int64

In [44]:
german_dataset.reset_format()

## Remap the labels

In [48]:
german_dataset = german_dataset.rename_column("stars", "labels")

In [49]:
label_mapping = {idx+1:idx for idx in range(5)}
label_mapping

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4}

In [50]:
def map_labels(examples):
    return {"labels": label_mapping[examples["labels"]]}

In [51]:
german_dataset = german_dataset.map(map_labels)

  0%|          | 0/200000 [00:00<?, ?ex/s]

  0%|          | 0/5000 [00:00<?, ?ex/s]

  0%|          | 0/5000 [00:00<?, ?ex/s]

## Zero-shot baseline

In [52]:
zeroshot_classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli", device=0)

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [54]:
zeroshot_classifier("Ich liebe dieses Buch!", candidate_labels=[0,1,2,3,4])

{'sequence': 'Ich liebe dieses Buch!',
 'labels': [3, 4, 0, 2, 1],
 'scores': [0.2328941971063614,
  0.2296580672264099,
  0.21016642451286316,
  0.19636668264865875,
  0.13091468811035156]}

In [55]:
def compute_zeroshot_preds(examples):
    preds = zeroshot_classifier(examples["review_body"], candidate_labels=[0,1,2,3,4])
    return {"zeroshot_prediction": preds["labels"][0]}

In [56]:
german_test_dataset = german_dataset["test"].map(compute_zeroshot_preds)
german_test_dataset

  0%|          | 0/5000 [00:00<?, ?ex/s]

/home/lewis/miniconda3/envs/hf/lib/python3.9/site-packages/transformers/pipelines/base.py:899: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [57]:
german_test_dataset["zeroshot_prediction"][:10]

[2, 0, 4, 0, 1, 0, 2, 2, 2, 2]

In [58]:
german_test_dataset["labels"][0]

0

In [59]:
mean_absolute_error(german_test_dataset["labels"], german_test_dataset["zeroshot_prediction"])

1.2926

## Tokenization

In [60]:
model_checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [61]:
def tokenize_reviews(examples):
    return tokenizer(examples["review_body"], truncation=True, max_length=512)

In [62]:
tokenized_dataset = german_dataset.map(tokenize_reviews, batched=True)

  0%|          | 0/200 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

## Load model

In [63]:
num_labels = 5
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

## Create metrics

In [64]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"MAE": mean_absolute_error(labels, predictions)}

## Create Trainer

In [67]:
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [65]:
model_name = model_checkpoint.split("/")[-1]
batch_size = 16
num_train_epochs = 3

num_train_samples = 500
train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(num_train_samples))
logging_steps = len(train_dataset) // (batch_size * num_train_epochs)

args = TrainingArguments(
    f"{model_name}-finetuned-marc-{num_train_samples}-samples",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    logging_steps=logging_steps,
    push_to_hub=True,
)

In [68]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/home/lewis/git/workshops/nlp-zurich/xlm-roberta-base-finetuned-marc-500-samples is already a clone of https://huggingface.co/lewtun/xlm-roberta-base-finetuned-marc-500-samples. Make sure you pull the latest changes with `repo.git_pull()`.


In [69]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: reviewer_id, product_id, review_title, review_body, review_id, product_category, language.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16


{'eval_loss': 1.6235263347625732,
 'eval_MAE': 2.0,
 'eval_runtime': 17.1764,
 'eval_samples_per_second': 291.098,
 'eval_steps_per_second': 18.223}

In [70]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: reviewer_id, product_id, review_title, review_body, review_id, product_category, language.
***** Running training *****
  Num examples = 500
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 160


Epoch,Training Loss,Validation Loss,Mae
1,1.614900,1.603234,1.590600
2,1.596700,1.589962,1.249600
3,1.567500,1.485550,1.149600
4,1.438200,1.366122,0.845400
5,1.300900,1.327512,0.874600


The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: reviewer_id, product_id, review_title, review_body, review_id, product_category, language.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16
Saving model checkpoint to xlm-roberta-base-finetuned-marc-500-samples/checkpoint-32
Configuration saved in xlm-roberta-base-finetuned-marc-500-samples/checkpoint-32/config.json
Model weights saved in xlm-roberta-base-finetuned-marc-500-samples/checkpoint-32/pytorch_model.bin
tokenizer config file saved in xlm-roberta-base-finetuned-marc-500-samples/checkpoint-32/tokenizer_config.json
Special tokens file saved in xlm-roberta-base-finetuned-marc-500-samples/checkpoint-32/special_tokens_map.json
tokenizer config file saved in xlm-roberta-base-finetuned-marc-500-samples/tokenizer_config.json
Special tokens file saved in xlm-roberta-base-finetuned-marc-500-samples/special_tokens_m

TrainOutput(global_step=160, training_loss=1.505726170539856, metrics={'train_runtime': 219.5709, 'train_samples_per_second': 11.386, 'train_steps_per_second': 0.729, 'total_flos': 232485434971824.0, 'train_loss': 1.505726170539856, 'epoch': 5.0})

In [71]:
trainer.push_to_hub(commit_message="Training complete")

Saving model checkpoint to xlm-roberta-base-finetuned-marc-500-samples
Configuration saved in xlm-roberta-base-finetuned-marc-500-samples/config.json
Model weights saved in xlm-roberta-base-finetuned-marc-500-samples/pytorch_model.bin
tokenizer config file saved in xlm-roberta-base-finetuned-marc-500-samples/tokenizer_config.json
Special tokens file saved in xlm-roberta-base-finetuned-marc-500-samples/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 32.0k/1.04G [00:00<?, ?B/s]

Upload file runs/Oct12_16-57-03_vorace/events.out.tfevents.1634050856.vorace: 100%|##########| 8.01k/8.01k [00…

remote: error: cannot lock ref 'refs/heads/main': is at cc8d97f269f272d245b499730226a5e45c790897 but expected 1ad385c8a07abc1ca653d5ac11b1e91586cb4163        
To https://huggingface.co/lewtun/xlm-roberta-base-finetuned-marc-500-samples
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/lewtun/xlm-roberta-base-finetuned-marc-500-samples'



OSError: remote: error: cannot lock ref 'refs/heads/main': is at cc8d97f269f272d245b499730226a5e45c790897 but expected 1ad385c8a07abc1ca653d5ac11b1e91586cb4163        
To https://huggingface.co/lewtun/xlm-roberta-base-finetuned-marc-500-samples
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/lewtun/xlm-roberta-base-finetuned-marc-500-samples'


## Zero-shot cross-lingual evaluation

In [72]:
def evaluate_corpus(lang):
    dataset = load_dataset(dataset_name, lang, split="test")
    dataset = dataset.rename_column("stars", "labels")
    dataset = dataset.map(map_labels)
    tokenized_dataset = dataset.map(tokenize_reviews, batched=True)
    preds = trainer.evaluate(eval_dataset=tokenized_dataset)
    return {"MAE": preds["eval_MAE"]}

In [73]:
evaluate_corpus("en")

Reusing dataset amazon_reviews_multi (/data/.cache/hf/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)


  0%|          | 0/5000 [00:00<?, ?ex/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: reviewer_id, product_id, review_title, review_body, review_id, product_category, language.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16


{'MAE': 0.874}

In [74]:
evaluate_corpus("en")

Reusing dataset amazon_reviews_multi (/data/.cache/hf/datasets/amazon_reviews_multi/fr/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)


  0%|          | 0/5000 [00:00<?, ?ex/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: reviewer_id, product_id, review_title, review_body, review_id, product_category, language.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16


{'MAE': 0.8742}

In [75]:
classifier = pipeline("text-classification", model=trainer.model, tokenizer=trainer.tokenizer, device=0)

In [76]:
classifier("I love this book!")

[{'label': 'LABEL_3', 'score': 0.30068162083625793}]

In [77]:
classifier("Ich hasse dieses Buch!")

[{'label': 'LABEL_0', 'score': 0.372832715511322}]

In [78]:
classifier("J'adore ce livre")

[{'label': 'LABEL_3', 'score': 0.2572416365146637}]